In [51]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [44]:
# Set the path to the folders containing the images
train_dir = './training_data/'

# Define the image size
img_height = 224
img_width = 224

# Define the batch size
batch_size = 4

# Define the number of classes
num_classes = 2


In [58]:
# Create a training dataset from the images in the training folder
(train_ds, validation_ds) = keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Print the number of batches in the training and validation datasets
print("Number of batches in training dataset: %d" %
      tf.data.experimental.cardinality(train_ds))
print("Number of batches in validation dataset: %d" %
      tf.data.experimental.cardinality(validation_ds))


Found 20 files belonging to 2 classes.
Using 16 files for training.
Using 4 files for validation.
Number of batches in training dataset: 4
Number of batches in validation dataset: 1


In [59]:
train_ds
classes = train_ds.class_names

In [60]:
# Create the model
model = keras.Sequential([
    keras.layers.experimental.preprocessing.Rescaling(
        1./255, input_shape=(img_height, img_width, 3)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3),
                  activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Fit the model to the training dataset
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    batch_size=batch_size,
    epochs=10
)


Epoch 1/10
4/4 [==============================] - 2s 287ms/step - loss: 18.5908 - accuracy: 0.5000 - val_loss: 6.2853 - val_accuracy: 0.5000
Epoch 2/10
4/4 [==============================] - 1s 216ms/step - loss: 3.7846 - accuracy: 0.5000 - val_loss: 1.0542 - val_accuracy: 0.5000
Epoch 3/10
4/4 [==============================] - 1s 105ms/step - loss: 0.7829 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/10
4/4 [==============================] - 1s 96ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/10
4/4 [==============================] - 3s 723ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/10
4/4 [==============================] - 1s 159ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/10
4/4 [==============================] - 2s 103ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 8/10
4/4 [====

In [77]:
def prediction_to_class(p, classes):
    for (i, x) in enumerate(classes):
        if p < (i+1) * (1 / len(classes)):
            return x;
    return classes[-1]

In [85]:
test_img_path = './training_data/healthy/1.bmp'

# Load a new image
new_image = tf.keras.preprocessing.image.load_img(
    test_img_path, target_size=(img_height, img_width))

# Preprocess the image
new_image = tf.keras.preprocessing.image.img_to_array(new_image)
new_image = tf.expand_dims(new_image, axis=0)

# Make a prediction with the model
prediction = model.predict(new_image)
prediction = prediction_to_class(prediction, classes)
prediction

1/1 [==============================] - 0s 18ms/step


'infected'